In [1]:
from sympy import *
from sympy.printing.c import C99CodePrinter


g = 9.81

def f_continuous(X,U,W):
    x,y,z,vx,vy,vz,phi,theta,psi,lx,ly,lz,lp,lq,lr = X
    ax,ay,az,p,q,r = U
    wx,wy,wz,wp,wq,wr = W
    
    # rotation matrix
    Rx = Matrix([[1, 0, 0], [0, cos(phi), -sin(phi)], [0, sin(phi), cos(phi)]])
    Ry = Matrix([[cos(theta), 0, sin(theta)],[0, 1, 0],[-sin(theta), 0, cos(theta)]])
    Rz = Matrix([[cos(psi), -sin(psi), 0],[sin(psi), cos(psi), 0], [0, 0, 1]])
    R = Rz*Ry*Rx
    a_NED = R*Matrix([ax-lx-wx,ay-ly-wy,az-lz-wz])
    
    return Matrix([
        vx,
        vy,
        vz,
        a_NED[0],
        a_NED[1],
        a_NED[2] + g,
        (p-lp-wp) + (q-lq-wq)*sin(phi)*tan(theta) + (r-lr-wr)*cos(phi)*tan(theta),
        (q-lq-wq)*cos(phi) - (r-lr-wr)*sin(phi),
        (q-lq-wq)*sin(phi)/cos(theta) + (r-lr-wr)*cos(phi)/cos(theta),
        0,0,0,0,0,0
    ])

def h(X):
    x,y,z,vx,vy,vz,phi,theta,psi,lx,ly,lz,lp,lq,lr = X
    return Matrix([x,y,z,phi,theta,psi])

# X=Matrix(symbols('x y z v_x v_y v_z phi theta psi l_x l_y l_z l_p l_q l_r'))
X=Matrix(symbols('X[0] X[1] X[2] X[3] X[4] X[5] X[6] X[7] X[8] X[9] X[10] X[11] X[12] X[13] X[14]'))
# U=Matrix(symbols('a_x a_y a_z p q r'))
U=Matrix(symbols('U[0] U[1] U[2] U[3] U[4] U[5]'))
# W=Matrix(symbols('w_x w_y w_z w_p w_q w_r'))
W=Matrix(symbols('W[0] W[1] W[2] W[3] W[4] W[5]'))

dt=symbols('dt')

# discretized dynamics
f = X + f_continuous(X,U,W)*dt

# output function
h = h(X)

# matrices
F = f.jacobian(X)
L = f.jacobian(W)
H = h.jacobian(X)

# substitute W with 0
f = f.subs([(w,0) for w in W])
F = F.subs([(w,0) for w in W])
L = L.subs([(w,0) for w in W])
H = H.subs([(w,0) for w in W])

# extra matrices
P = Matrix([[symbols(f'P[{i}][{j}]') for j in range(len(X))] for i in range(len(X))])
Q = Matrix([[symbols(f'Q_diag[{i}]') if i==j else '0' for j in range(len(U))] for i in range(len(U))])
R = Matrix([[symbols(f'R_diag[{i}]') if i==j else '0' for j in range(len(h))] for i in range(len(h))])
Z = Matrix(symbols('Z[0] Z[1] Z[2] Z[3] Z[4] Z[5]'))

In [2]:
''' We are gonna generate C code that implements a Extended Kalman filter (EKF)
The kalman filter consists of the following global vars
- X the ekf state
- P the ekf covariance
- Q the process noise covariance
- R the observation noise covariance
and the following functions
- ekf_init() initializes the ekf with initial guess for X and P
- ekf_predict(U) prediction step based on input U
- ekf_update(Z) update step based on measurement Z
'''

# create c_code folder
import os
if not os.path.exists('c_code'):
    os.makedirs('c_code')

# create header file to write to
with open('c_code/ekf.h','w') as file:
    file.write('''
#ifndef EKF_H
#define EKF_H

#include <stdint.h>

#define N_STATES %d
#define N_INPUTS %d
#define N_MEASUREMENTS %d

extern float X[N_STATES];
extern float P[N_STATES][N_STATES];
extern float Q_diag[N_INPUTS];
extern float R_diag[N_MEASUREMENTS];

void ekf_predict(float U[N_INPUTS], float dt);
void ekf_update(float Z[N_MEASUREMENTS]);

#endif
''' % (len(X),len(U),len(h)))

# Write the source file
with open('c_code/ekf.c','w') as file:
    file.write('''
#include "ekf.h"
#include <math.h>

float X[N_STATES];
float P[N_STATES][N_STATES];
float Q_diag[N_INPUTS];
float R_diag[N_MEASUREMENTS];
               
/** Calculate inverse of any n x n matrix (passed as C array) o = mat^-1
Algorithm verified with Matlab.
Thanks to: https://www.quora.com/How-do-I-make-a-C++-program-to-get-the-inverse-of-a-matrix-100-X-100
*/
void float_mat_invert(float **o, float **mat, int n)
{
  int i, j, k;
  float t;
  float a[n][2 * n];

  // Append an identity matrix on the right of the original matrix
  for (i = 0; i < n; i++) {
    for (j = 0; j < 2 * n; j++) {
      if (j < n) {
        a[i][j] = mat[i][j];
      } else if ((j >= n) && (j == i + n)) {
        a[i][j] = 1.0;
      } else {
        a[i][j] = 0.0;
      }
    }
  }

  // Do the inversion
  for (i = 0; i < n; i++) {
    t = a[i][i]; // Store diagonal variable (temp)

    for (j = i; j < 2 * n; j++) {
      a[i][j] = a[i][j] / t; // Divide by the diagonal value
    }

    for (j = 0; j < n; j++) {
      if (i != j) {
        t = a[j][i];
        for (k = 0; k < 2 * n; k++) {
          a[j][k] = a[j][k] - t * a[i][k];
        }
      }
    }
  }

  // Cut out the identity, which has now moved to the left side
  for (i = 0 ; i < n ; i++) {
    for (j = n; j < 2 * n; j++) {
      o[i][j - n] = a[i][j];
    }
  }
}

void ekf_predict(float U[N_INPUTS], float dt) {
    // [1] state prediction X_pred = f(X,U,dt)
    float X_pred[N_STATES];
'''
    )
    for i in range(len(X)):
        file.write('    X_pred[%d] = ' % i + ccode(f[i]) + ';\n')
    file.write('''
    // [2] covariance prediction P = F*P*F.T + L*Q*L.T
    float P_pred[N_STATES][N_STATES]; \n''')
    # symbolic expression for P_pred
    P_pred = F*P*F.T + L*Q*L.T
    for i in range(len(X)):
        for j in range(len(X)):
            file.write('    P_pred[%d][%d] = ' % (i,j) + ccode(P_pred[i,j]) + ';\n')
    file.write('''
    // [3] overwrite state and covariance
    int i,j;
    for(i=0;i<N_STATES;i++) {
        X[i] = X_pred[i];
        for(j=0;j<N_STATES;j++) {
            P[i][j] = P_pred[i][j];
        }
    }
}
''')
    
    file.write('''
void ekf_update(float Z[N_MEASUREMENTS]) {
    // Residual covariance: S = H*P*H.T + R
    float S[N_MEASUREMENTS][N_MEASUREMENTS] = { \n''')
    # symbolic expression for S
    S = H*P*H.T + R
    for i in range(len(h)):
        file.write('        {')
        for j in range(len(h)):
            file.write(ccode(S[i,j]) + ',')
        if i>=len(h)-1:
            file.write('}\n')
        else:
            file.write('},\n')
    file.write('    };')
    file.write('''
    // Inverse of S
    float S_inv[N_MEASUREMENTS][N_MEASUREMENTS];
    float_mat_invert(S_inv,S,N_MEASUREMENTS);
               
''')
    # symbolic expression for S_inv
    S_inv = Matrix([[symbols(f'S_inv[{i}][{j}]') for j in range(len(U))] for i in range(len(U))])
    K = P*H.T*S_inv
    X_up = X + K*(Z - h)
    P_up = (eye(len(X)) - K*H)*P
    file.write('    // Kalman gain: K = P*H.T*S_inv \n')
    file.write('    // [1] state update X_up = X + K*(Z - h(X)) \n')
    file.write('    float X_up[N_STATES];\n')
    for i in range(len(X)):
        file.write('    X_up[%d] = ' % i + ccode(X_up[i]) + ';\n')
    
    file.write('\n')
    file.write('    // [2] covariance update P_up = (I - K*H)*P \n')
    file.write('    float P_up[N_STATES][N_STATES];\n')
    for i in range(len(X)):
        for j in range(len(X)):
            file.write('    P_up[%d][%d] = ' % (i,j) + ccode(P_up[i,j]) + ';\n')
    file.write('''
    // [3] overwrite state and covariance
    int i,j;
    for(i=0;i<N_STATES;i++) {
        X[i] = X_up[i];
        for(j=0;j<N_STATES;j++) {
            P[i][j] = P_up[i][j];
        }
    }
}''')
            